In [1]:
import os
import ast
import time
import spacy
import openai
import numpy as np
import pandas as pd 
from sentence_transformers import util
from sklearn.metrics.pairwise import cosine_similarity
from vertexai.language_models import TextEmbeddingModel
from vertexai.language_models import TextGenerationModel
from vertexai.preview.language_models import *#ChatModel, InputOutputTextPair

2023-09-25 11:58:39.316169: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-09-25 11:58:40.929420: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [2]:
openai.api_key = "" # Replace with OPENAI API Key

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "" # Replace with GCP Credentials JSON File Path

# Generate Testing Variations from Question/Answer CSV using OPENAI API

In [8]:
def generate_variations_openai(prompt):
    full_prompt = f"Generate a 'single' unique variation for the question below and return another single question without changing the semantics and the meaning, only the wording should be different: \"{prompt}\""
    
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=full_prompt,
    temperature=0.1
    )

    output = list()
    for k in response['choices']:
        output.append(k['text'].strip())
    return ''.join(output)

In [168]:
def generate_variations_vertex(prompt):
    model = TextGenerationModel.from_pretrained("text-bison@001")
    
    full_prompt = f"Generate a 'single' unique variation for the question below and return another single question without changing the semantics and the meaning, only the wording should be different: \"{prompt}\""
    
    response = model.predict(prompt=full_prompt, temperature=0.4)

    return (response.text.strip())

### Saving Variations locally

In [ ]:
count = 1
response_list =[]
answer_list=[]

for i in range(0, len(df), 6):
    # print(f"{i}:  {df['Question'][i]}")
    # response = generate_variations_vertex(df['Question'][i])
    response = generate_variations_openai(df['Question'][i])
    
    print(f"{i}: {response}")

    response_list.append(response)
    answer_list.append(df['Answer'][i])

In [155]:
# answer_list

In [13]:
testing_df = pd.DataFrame({'Question': response_list, 'Answer': answer_list})
testing_df.head()
testing_df.to_csv('Test_Questions.csv', index=False)

## Evaluating the Chat Bot

In [15]:
nlp = spacy.load("en_core_web_sm")

In [16]:
def preprocess_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc if not token.is_punct])

### Answering the user question

In [24]:
def get_most_similar_question(user_question, question_embeddings_df):
    user_question = preprocess_text(user_question)
    model = TextEmbeddingModel.from_pretrained("textembedding-gecko@001")
    user_question_embedding = model.get_embeddings([user_question])
    for embedding in user_question_embedding:
        user_question_embedding = embedding.values
    
    question_embeddings = question_embeddings_df['Embedding']
    sim_score = []
    for i in question_embeddings:
        sim_score.append(util.pytorch_cos_sim(user_question_embedding, i).item())
    
    threshhold = 0.80
    
    max_sim_score = max(sim_score)

    # print(max_sim_score)
    
    if max_sim_score >= threshhold:
        sim_arg = sim_score.index(max_sim_score)
        return embedding_df['Answer'][sim_arg], max_sim_score
    else:
        var = "Sorry, Couldn't find an answer"
        return var, max_sim_score

### Applying text cleaning

In [ ]:
df = pd.read_csv("Path/To/question-answers.csv")
embedding_df['Embedding'] = embedding_df['Embedding'].apply(ast.literal_eval)
embedding_df['Question'] = embedding_df['Question'].str.replace(r'[^\x00-\x7F]+', '', regex=True)
embedding_df['Answer'] = embedding_df['Answer'].str.replace(r'[^\x00-\x7F]+', '', regex=True)
embedding_df.head()

### Evaluating each question and storing the results in a txt file

In [28]:
import time
import pandas as pdfdf

def evaluate_chatbot(test_dataframe, embedding_df, output_file_path):
    correct_answers = 0
    unable_to_decide = 0
    total_questions = len(test_dataframe)

    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        for index, row in test_dataframe.iterrows():
            test_question = row['Question']
            ground_truth_answer = row['Answer']
    
            # print(test_question)
    
            response, score = get_most_similar_question(test_question, embedding_df)
    
            if response == ground_truth_answer:
                correct_answers += 1
                result = "Correct"
            elif response == "Sorry, Couldn't find an answer":
                result = "Unable to Decide"
            else:
                result = "Wrong"
                # print("\nWrong Answer")
            
            print(f"\n\nQuestion: {test_question}\n")
            print(f"Chatbot's Response: {response}\n")
            print(f"Ground Truth Answer: {ground_truth_answer}\n")
            print(f"Score: {score}\n")
            print(f"Result: {result}\n\n")

            output_file.write(f"\n\nQuestion: {test_question}\n")
            output_file.write(f"Chatbot's Response: {response}\n")
            output_file.write(f"Ground Truth Answer: {ground_truth_answer}\n")
            output_file.write(f"Score: {score}\n")
            output_file.write(f"Result: {result}\n\n")
            
        accuracy = (correct_answers / total_questions) * 100
    return accuracy

In [ ]:
evaluate_chatbot(testing_df, embedding_df, "evaluation_results.txt")